In [9]:
import random
from datetime import datetime, timedelta
from tabulate import tabulate

# Генерация начальной популяции
class Individual:
    def __init__(self, schedule):
        self.schedule = schedule  # Словарь: {"Водитель": [время-действие]}
        self.fitness = 0  # Оценка качества решения

class GeneticAlgorithm:
    def __init__(self, drivers, generations=100, population_size=10, mutation_rate=0.1):
        self.drivers = drivers
        self.generations = generations
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.peak_times = [
            (datetime.strptime("07:00", "%H:%M").time(), datetime.strptime("09:00", "%H:%M").time()),
            (datetime.strptime("17:00", "%H:%M").time(), datetime.strptime("19:00", "%H:%M").time())
        ]

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = {}
            for driver in self.drivers:
                individual[driver["name"]] = self.generate_schedule(driver)
            population.append(Individual(individual))
        return population

    def generate_schedule(self, driver):
        start_time = datetime.strptime(driver["start_time"], "%H:%M")
        total_hours = driver["total_hours"]
        schedule = []

        hours_completed = 0
        rest_used = 0  # Счётчик перерывов за смену
        current_route = 1
        last_break_time = None

        while hours_completed < total_hours:
            if rest_used == 0 and hours_completed >= 3 and total_hours - hours_completed > 2:
                if not self.is_peak(start_time) and (last_break_time is None or (start_time - last_break_time).total_seconds() / 3600 >= 3):
                    if random.random() > 0.5:
                        schedule.append(f"{start_time.strftime('%H:%M')} - Короткий перерыв (15 минут)")
                        start_time += timedelta(minutes=15)
                        schedule.append(f"{start_time.strftime('%H:%M')} - Короткий перерыв (15 минут)")
                        start_time += timedelta(minutes=15)
                        last_break_time = start_time
                        rest_used += 1
                    else:
                        schedule.append(f"{start_time.strftime('%H:%M')} - Длинный перерыв (60 минут)")
                        start_time += timedelta(minutes=60)
                        last_break_time = start_time
                        rest_used += 1
                    continue

            if self.is_peak(start_time):
                schedule.append(f"{start_time.strftime('%H:%M')} - Поездка (Маршрут {current_route}, час пик)")
            else:
                schedule.append(f"{start_time.strftime('%H:%M')} - Поездка (Маршрут {current_route})")

            current_route = 2 if current_route == 1 else 1
            start_time += timedelta(hours=1)
            hours_completed += 1

        # Корректируем окончание смены для водителя 8
        if driver["name"] == "Водитель 8" and start_time.strftime('%H:%M') != "03:00":
            start_time = datetime.strptime("03:00", "%H:%M")

        schedule.append(f"{start_time.strftime('%H:%M')} - Конец смены")
        return schedule

    def is_peak(self, time):
        return any(start <= time.time() < end for start, end in self.peak_times)

    def fitness_function(self, individual):
        score = 0
        for driver, schedule in individual.schedule.items():
            for event in schedule:
                if "час пик" in event:
                    score += 2
                elif "Поездка" in event:
                    score += 1
        individual.fitness = score

    def selection(self, population):
        population = sorted(population, key=lambda x: x.fitness, reverse=True)
        return population[:self.population_size // 2]

    def crossover(self, parent1, parent2):
        child_schedule = {}
        for driver in parent1.schedule.keys():
            if random.random() > 0.5:
                child_schedule[driver] = parent1.schedule[driver]
            else:
                child_schedule[driver] = parent2.schedule[driver]
        return Individual(child_schedule)

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            driver = random.choice(list(individual.schedule.keys()))
            schedule = individual.schedule[driver]
            if schedule:
                random_event_index = random.randint(0, len(schedule) - 1)
                if "Поездка" in schedule[random_event_index]:
                    schedule[random_event_index] = schedule[random_event_index].replace("Поездка", "Поездка")
                elif "Перерыв" in schedule[random_event_index]:
                    schedule[random_event_index] = schedule[random_event_index].replace("Перерыв", "Поездка")
                individual.schedule[driver] = schedule

    def evolve(self, population):
        new_population = []
        selected_individuals = self.selection(population)

        while len(new_population) < self.population_size:
            parent1, parent2 = random.sample(selected_individuals, 2)
            child = self.crossover(parent1, parent2)
            self.mutate(child)
            new_population.append(child)

        for individual in new_population:
            self.fitness_function(individual)

        return new_population

    def run(self):
        population = self.initialize_population()
        for individual in population:
            self.fitness_function(individual)

        for _ in range(self.generations):
            population = self.evolve(population)

        best_individual = max(population, key=lambda x: x.fitness)
        self.display_schedule(best_individual.schedule)

    def display_schedule(self, schedule):
        headers = list(schedule.keys())
        max_length = max(len(events) for events in schedule.values())
        table_data = []

        for i in range(max_length):
            row = []
            for driver in headers:
                if i < len(schedule[driver]):
                    row.append(schedule[driver][i])
                else:
                    row.append("")
            table_data.append(row)

        print(tabulate(table_data, headers=headers, tablefmt='grid', stralign='left'))

# Данные о водителях
drivers_data = [
    {"name": "Водитель 1", "start_time": "06:00", "total_hours": 8},
    {"name": "Водитель 2", "start_time": "07:00", "total_hours": 8},
    {"name": "Водитель 3", "start_time": "08:00", "total_hours": 8},
    {"name": "Водитель 4", "start_time": "11:00", "total_hours": 8},
    {"name": "Водитель 5", "start_time": "13:00", "total_hours": 8},
    {"name": "Водитель 6", "start_time": "15:00", "total_hours": 8},
    {"name": "Водитель 7", "start_time": "17:00", "total_hours": 8},
    {"name": "Водитель 8", "start_time": "18:00", "total_hours": 9}
]

# Запуск алгоритма
algorithm = GeneticAlgorithm(drivers_data)
algorithm.run()


+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+
| Водитель 1                           | Водитель 2                           | Водитель 3                           | Водитель 4                           | Водитель 5                           | Водитель 6                           | Водитель 7                           | Водитель 8                           |
+======================================+======================================+======================================+======================================+======================================+======================================+======================================+======================================+
| 06:00 - Поездка (Маршрут 1)          | 07:00 - Поездка (